In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['jpg']
import matplotlib
matplotlib.figure.Figure.__repr__ = lambda self: (
    f"<{self.__class__.__name__} size {self.bbox.size[0]:g}"
    f"x{self.bbox.size[1]:g} with {len(self.axes)} Axes>")

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)
import pandas as pd
import scipy.cluster.hierarchy as sch
import scipy
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

In [ ]:
def plot_fit_residual(x, y, yp):

    # Model
    fig, axes  = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
    axes[0].plot(x.ravel(), y.ravel(), "k.",  label="Podatki")
    axes[0].plot(x.ravel(), yp.ravel(), "g-", label="Model h(x)")
    axes[0].set_xlabel("x")
    axes[0].set_ylabel("y")
    axes[0].legend(loc=4)

    # Ostanki
    r = pearsonr(y.ravel(), y.ravel()-yp.ravel())[0]
    axes[1].plot(y.ravel(), y.ravel()-yp.ravel(), "k.", label="Ostanek")
    axes[1].set_xlabel("y")
    axes[1].set_ylabel("y-h(x)")
    axes[1].set_title("Graf ostankov, R=%.3f" % r)
    axes[1].legend(loc=4)
    plt.show()
    
def epsilon(data, regija, letoRegija):
    x = []
    y = []
    for key in data:
        summ = []
        for i in data[key]:
            if i != letoRegija and regija in i:
                summ.append(float(data[key][i]))
        x.append([sum(summ) / len(summ)])
        y.append(float(data[key][regija]))
    return [x, y]    

def ixs(data, regija):
    x = []
    y = []
    for key in data:
        summ = []
        for i in data[key]:
            if regija in i:
                summ.append(float(data[key][i]))
        x.append([sum(summ) / len(summ)])
    return x  

def addToData(data, dataPredicted, leto):
    z = 0
    for key in data:
        for i in dataPredicted:
            string = '"' + str(leto) + ' ' + i + '"'
            data[key][string] = dataPredicted[i][z]
        z += 1
    return data

def kulturaRegijaLeto(data, regija, leto):
    maxKultura = ["0", "soncek"]
    for key in data:
        for i in data[key]:
            if regija in i and str(leto) in i:
                if float(data[key][i]) > float(maxKultura[0]):
                    maxKultura[0] = data[key][i]
                    maxKultura[1] = key
    return maxKultura[1]  

def epsilonCene(data, kulture, leto):
    x = []
    y = []
    for j in kulture:
        summ = []
        for key in data:
            for i in data[key]:
                if key != leto:
                    if i == j:
                        summ.append(float(data[key][i]))
        x.append([sum(summ) / len(summ)])
        y.append(float(data[leto][j]))
    return [x, y]   

In [ ]:
from csv import DictReader

podatki = []
i = 0

readerRatings = DictReader(open('1502410S_20210405-145226.csv', 'rt'))
for row in readerRatings:
    if i != 0:
        podatki.append(row)
    else:
        imenaKrajev = row
    i += 1


imena = []
stevilke = []

for key in podatki:
    imena.append(key["Povprečni pridelek (t/ha) pomembnejših kmetijskih kultur"])
    stevilke.append(key[None])

for key in range(len(stevilke)):
    for i in range(len(stevilke[0])):
        if stevilke[key][i] == "-":
            stevilke[key][i] = 0
        
podatki = {}

for i in range(len(imena)):
    podatki[imena[i]] = stevilke[i]


imenaKrajev = imenaKrajev["Povprečni pridelek (t/ha) pomembnejših kmetijskih kultur"].split(",")  
df = pd.DataFrame(podatki)
df.index = imenaKrajev
df

In [ ]:
string = '"' + str(2010) + ' SLOVENIJA"'

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1
x = [0,1,2,3,4,5,6,7,8,9]
y = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

plt.figure(figsize=(15,10))
plt.ylabel("t/ha")
plt.plot(povprecja, marker="o", label="povprecje Slovenije", linewidth=5)
plt.xticks(x,range(2010, 2020, 1))


povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Jugovzhodna Slovenija"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Jugovzhodna Slovenija")


povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Obalno-kraška"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Obalno-kraška")


povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Goriška"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Goriška")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Primorsko-notranjska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Primorsko-notranjska")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Gorenjska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Gorenjska")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Osrednjeslovenska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Osrednjeslovenska")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Posavska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Posavska")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Zasavska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Zasavska")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Savinjska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Savinjska")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Koroška"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Koroška")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Podravska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Podravska", color="yellow")

povprecja = []
leta = 0
letnice = 2010

for i in range(2010, 2020, 1):
    
    string = '"' + str(letnice) + ' Pomurska"'
    for i in range(len(df.loc[string])):
        leta += float(df.loc[string][i])

    leta = leta / len(df.loc[string])
    povprecja.append(leta)
    letnice += 1


plt.plot(povprecja, marker="o", label="povprecje Pomurska", color="black")


plt.legend(bbox_to_anchor=(0.7, -0.05))
plt.title("Povprečna pridelava živil letno po regijah")
plt.show()


df = df.drop(labels=['"2010 SLOVENIJA"', '"2010 Pomurska"', '"2010 Podravska"', '"2010 Koroška"', '"2010 Savinjska"', '"2010 Zasavska"', '"2010 Posavska"', '"2010 Jugovzhodna Slovenija"', '"2010 Osrednjeslovenska"', '"2010 Gorenjska"', '"2010 Primorsko-notranjska"', '"2010 Goriška"', '"2010 Obalno-kraška"', '"2011 SLOVENIJA"', '"2011 Pomurska"', '"2011 Podravska"', '"2011 Koroška"', '"2011 Savinjska"', '"2011 Zasavska"', '"2011 Posavska"', '"2011 Jugovzhodna Slovenija"', '"2011 Osrednjeslovenska"', '"2011 Gorenjska"', '"2011 Primorsko-notranjska"', '"2011 Goriška"', '"2011 Obalno-kraška"', '"2012 SLOVENIJA"', '"2012 Pomurska"', '"2012 Podravska"', '"2012 Koroška"', '"2012 Savinjska"', '"2012 Zasavska"', '"2012 Posavska"', '"2012 Jugovzhodna Slovenija"', '"2012 Osrednjeslovenska"', '"2012 Gorenjska"', '"2012 Primorsko-notranjska"', '"2012 Goriška"', '"2012 Obalno-kraška"'], axis=0)

for i in range(0, 39):
    del imenaKrajev[0]


In [ ]:

    


povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][3])
    povprecja.append(steviloLetno)
    letnice += 1

plt.figure(figsize=(15,10))
plt.plot(povprecja, marker="o", label="Krompir")
plt.ylabel("t/ha")
x = [0,1,2,3,4,5,6]
plt.xticks(x,range(2013, 2020, 1))

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][7])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Belo zelje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][9])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Jabolka v intenzivnih sadovnjakih")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][0])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Pšenica in pira")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][1])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Ječmen")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][2])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Koruza za zrnje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][4])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Buče za olje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][5])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Oljna ogrščica in repica")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][6])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Hmelj")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][8])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Grozdje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice) + ' SLOVENIJA"'
    steviloLetno = float(df.loc[string][10])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Breskve in nektarine v intenzivnih sadovnjakih", color="black")
plt.legend(bbox_to_anchor=(0.7, -0.05))
plt.title("Povprečna pridelava živil letno")
plt.show()

In [ ]:
prices = DictReader(open('1505000S.csv', 'rt'))

cene = []
i = 0
letaCene = []
for row in prices:
    if i != 0:
        cene.append(row)
    else:
        letaCene = row
    i += 1
    
imenaCene = []
stevilkeCene = []

for key in cene:
    imenaCene.append(key["Quantities and average prices of purchased agricultural products"])
    stevilkeCene.append(key[None])
    
cene = {}


for i in range(len(imenaCene)):
    cene[imenaCene[i]] = stevilkeCene[i]

letaCene = letaCene["Quantities and average prices of purchased agricultural products"].split(",")  
dfCene = pd.DataFrame(cene)
dfCene.index = letaCene

for i in range(len(dfCene["Pšenica in pira"].values)):
    dfCene["Pšenica in pira"].values[i] = float(dfCene["Pšenica in pira"].values[i]) / 1000
    
for i in range(len(dfCene["Ječmen"].values)):
    dfCene["Ječmen"].values[i] = float(dfCene["Ječmen"].values[i]) / 1000

for i in range(len(dfCene["Koruza za zrnje"].values)):
    dfCene["Koruza za zrnje"].values[i] = float(dfCene["Koruza za zrnje"].values[i]) / 1000
dfCene

In [ ]:
Average_price = {}
cene_po_panogah = []
flag = 0
for x in dfCene.index:
    if flag%2!=0:
        x = x.split()[0]
        Average_price[x] = {}
        for y in dfCene.columns:
            Average_price[x][y] = 0
    flag+=1
flag = 0
poljeSt = 0
poljestV = 0
for c in dfCene.values:
    if flag%2!=0:
        cene_po_panogah.append(c)
    flag+=1    

for a1 in Average_price.keys():
    for a2 in Average_price[a1].keys():
        Average_price[a1][a2]=cene_po_panogah[poljeSt][poljestV]
        poljestV+=1
    poljestV=0
    poljeSt+=1

regije = {'SLOVENIJA': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Pomurska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Podravska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Koroška': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Savinjska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Zasavska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Posavska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Jugovzhodna Slovenija': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Osrednjeslovenska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Gorenjska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Primorsko-notranjska': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Goriška': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}, 'Obalno-kraška': {"2013": {}, "2014": {},"2015": {},"2016": {},"2017": {},"2018": {},"2019": {}}}

for key in regije:
    for i in regije[key]:
        for j in range(len(df.loc['"' + str(i) + " " + str(key) + '"'])):
            regije[key][i][df.loc['"' + str(i) + " " + str(key) + '"'].keys()[j]] = df.loc['"' + str(i) + " " + str(key) + '"'][j]
            
regijePovprecno = {}
            
for key in regije:
    regijePovprecno[key] = {}
    for i in regije[key]:
        avg = 0
        for j in regije[key][i]:
            avg += (float(regije[key][i][j]) * float(Average_price['"' + str(i)][j]))
        avg = avg / len(regije[key][i])
        regijePovprecno[key][i] = avg
        

regijeCenaPovprecno = {}

for key in regijePovprecno:
    avg = 0
    for i in regijePovprecno[key]:
        avg += regijePovprecno[key][i]
    avg = avg / len(regijePovprecno[key])
    regijeCenaPovprecno[key] = avg
    
fig, ax = plt.subplots(figsize=(13,10))


podatki = {}

for key in regijeCenaPovprecno:
    if key != "SLOVENIJA":
        podatki[key] = regijeCenaPovprecno[key]
        
x = np.arange(len(podatki))
ax.barh(x, podatki.values())
ax.set_yticks(x)
plt.title("Povprečje regije od 2013 - 2019")
ax.set_yticklabels(podatki.keys())
ax.set_xticks(np.arange(6))
ax.set_xticklabels(range(0,6))

plt.show()

a=[regijePovprecno["SLOVENIJA"][key] for key in regijePovprecno["SLOVENIJA"]]
av = sum(a)/len(a)



for i in range(len(a)):
    a[i] = a[i] - av

povprecje = 0
idx=np.array([2013,2014,2015,2016,2017,2018,2019])
barve = ['green' if (x > 0) else 'red' for x in a ]
plt.bar(idx, a, color=barve, width=0.7, alpha = 1)
plt.xlabel('Bins')
plt.title("Odmik od povprečja zaslužka na t/ha letno v Sloveniji")
plt.ylabel('Frequency')
plt.axhline(y=0, color='darkgray', linestyle='-')

plt.show()

In [ ]:
profitablineKulture = {}
for j in range(11):
    povprecja = 0
    letnice = 2013
    for i in range(2013, 2020, 1):
        string = '"' + str(letnice) + ' SLOVENIJA"'
        povprecja += float(df.loc[string][j]) * float(Average_price['"' + str(letnice)][df.loc[string].keys()[j]])
        letnice += 1
    povprecja /= 7
    profitablineKulture[df.loc[string].keys()[j]] = povprecja   
    
profitablineKulture = sorted(profitablineKulture.items(), key=lambda item: item[1], reverse=False) 
podatki = {}
for key in profitablineKulture:
    podatki[key[0]] = key[1] 
print(profitablineKulture)
fig, ax = plt.subplots(figsize=(10,10))

x = np.arange(len(podatki))
ax.barh(x, podatki.values())
ax.set_yticks(x)
plt.title("Najbolj profitabilne kulture v Sloveniji")
ax.set_yticklabels(podatki.keys())
ax.set_xticks(np.arange(13))
ax.set_xticklabels(range(0,13))

In [ ]:
cenaNaKulturo = {}

for key in regije:
    cenaNaKulturo[key] = {}
    for i in regije[key]:
        cenaNaKulturo[key][i] = {}
        for j in regije[key][i]:
            cenaNaKulturo[key][i][j] = float(regije[key][i][j]) * float(Average_price['"' + str(i)][j])



pridelavaRegije = {}

for key in regije:
    for i in regije[key]:
        for j in regije[key][i]:
            if j not in pridelavaRegije:
                pridelavaRegije[j] = {}
                pridelavaRegije[j][key] = float(regije[key][i][j])
            else:
                if key not in pridelavaRegije[j]:
                    pridelavaRegije[j][key] = float(regije[key][i][j])
                else:
                    pridelavaRegije[j][key] += float(regije[key][i][j])
            
for key in pridelavaRegije:
    for i in pridelavaRegije[key]:
        pridelavaRegije[key][i] = round(round(pridelavaRegije[key][i], 1) / 7, 1)
        

primernaObmocjaSajenja = {}


for key in pridelavaRegije:
    primernaObmocjaSajenja[key] = max(pridelavaRegije[key], key=pridelavaRegije[key].get)
print("Najbolj primerne regije za vsako kulturo:")   
for key in primernaObmocjaSajenja:       
    print( key + " : " + primernaObmocjaSajenja[key])

In [ ]:
povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Krompir"])
    povprecja.append(steviloLetno)
    letnice += 1

plt.figure(figsize=(15,10))
plt.plot(povprecja, marker="o", label="Krompir")
plt.ylabel("€/(t/ha)")
x = [0,1,2,3,4,5,6]
plt.xticks(x,range(2013, 2020, 1))

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Belo zelje"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Belo zelje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Jabolka v intenzivnih sadovnjakih"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Jabolka v intenzivnih sadovnjakih")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Pšenica in pira"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Pšenica in pira")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Ječmen"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Ječmen")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Koruza za zrnje"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Koruza za zrnje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Buče za olje"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Buče za olje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Oljna ogrščica in repica"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Oljna ogrščica in repica")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Hmelj"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Hmelj")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Grozdje"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Grozdje")

povprecja = []
steviloLetno = 0
letnice = 2013

for i in range(2013, 2020, 1):
    
    string = '"' + str(letnice)
    steviloLetno = float(Average_price[string]["Breskve in nektarine v intenzivnih sadovnjakih"])
    povprecja.append(steviloLetno)
    letnice += 1


plt.plot(povprecja, marker="o", label="Breskve in nektarine v intenzivnih sadovnjakih", color="black")
plt.legend(bbox_to_anchor=(0.7, -0.05))
plt.title("Gibanje odkupnih cen kultur skozi leta")
plt.show()

In [ ]:
slovar = df.to_dict()
model = LinearRegression()
hx = {}
test = {}
data2019 = []
string = '"' + str(2014) + ' Obalno-kraška"'
kulture = df.loc[string].keys()
vseRegije = ["SLOVENIJA", "Pomurska", "Podravska", "Koroška", "Savinjska", "Zasavska", "Posavska", "Jugovzhodna Slovenija", "Osrednjeslovenska", "Gorenjska", "Primorsko-notranjska", "Goriška", "Obalno-kraška"]
for key in vseRegije:
    for i in imenaKrajev:
        if key in i and "2019" not in i:
            xy = epsilon(slovar, i, key)
            y = xy[1]
            x = xy[0]
            model.fit(x, y)
        elif key in i and "2019" in i:
            data2019 = []
            for j in slovar:
                for z in slovar[j]:
                    if key in z and "2019" in z:
                        data2019.append(slovar[j][z])
    test[key] = data2019
    hx[key] = model.predict(ixs(slovar, key)).round(1)
    
string = '"' + str(2014) + ' Obalno-kraška"'

dfNapoved = pd.DataFrame(hx)
dfNapoved.index = df.loc[string].keys()
dfNapoved = dfNapoved.T
dfNapoved


In [ ]:
dfNapoved2019 = pd.DataFrame(test)
dfNapoved2019.index = df.loc[string].keys()
dfNapoved2019 = dfNapoved2019.T
dfNapoved2019


In [ ]:
for j in range(2020, 2031, 1):
    for key in vseRegije:
        hx[key] = model.predict(ixs(slovar, key))
    slovar = addToData(slovar, hx, j)

print(slovar)

In [ ]:
for j in range(2013, 2020, 1):      
    print(kulturaRegijaLeto(slovar, "Goriška", j))

In [ ]:
counter = 0
model2 = LinearRegression()
string = '"2014 Average price (EUR)"'
kulture = dfCene.loc[string].keys()
for key in Average_price:
    if "2020" not in key:
        xy = epsilonCene(Average_price, kulture, key)
        y = xy[1]
        x = xy[0]
        model2.fit(x, y)
hx = model2.predict(x).round(2)
    
print(hx)
test = []
for key in Average_price:
    for i in Average_price[key]:
        if "2020" in key:
            test.append(float(Average_price[key][i]))
print(test)     

dfCeneNapoved = pd.DataFrame(hx)
dfCeneNapoved.index = kulture
dfCeneNapoved

In [ ]:
dfCeneNapoved2020 = pd.DataFrame(np.array(test).round(2))
dfCeneNapoved2020.index = kulture
dfCeneNapoved2020